<a href="https://colab.research.google.com/github/ArdeshirB/DMML/blob/main/Map-CC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import folium
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import ipywidgets as widgets
from IPython.display import display


# Convert your suburb data into a pandas dataframe
suburbs_data = {
    'Suburb Name': ['Hawthorn', 'Coorparoo', 'Eatons Hill', 'Ellenbrook North', 'Morayfield',
                    'Altona North', 'Cairns', 'North Lakes', 'Keperra', 'Annerley', 'Ballina'],
    'Latitude': [-37.8267, -27.4949, -27.3394, -31.7756, -27.0988, -37.8425, -16.9186, -27.2406,
                 -27.4285, -27.4981, -28.8667],
    'Longitude': [145.036, 153.0617, 152.9531, 116.0046, 152.9524, 144.8569, 145.7781, 153.0145,
                  152.9476, 153.0313, 153.5667]
}
suburbs_df = pd.DataFrame(suburbs_data)

# We'll adjust the parsing process to correctly get the values
date_raised = []
location = []
complaint_type = []

data_str = """
44961	Hawthorn	Packaging
44963	Coorparoo	Seal Integrity
44964	Eatons Hill	Recipe Control
44978	Ellenbrook North	Recipe Control
44979	Morayfield	Seal Integrity
44981	Morayfield	Seal Integrity
44982	Morayfield	Seal Integrity
44985	Morayfield	Seal Integrity
44987	Morayfield	Seal Integrity
44988	Morayfield	Seal Integrity
44988	Morayfield	Seal Integrity
44988	Eatons Hill	Seal Integrity
44988	Eatons Hill	Seal Integrity
44988	Morayfield	Seal Integrity
44989	Altona North	Seal Integrity
44992	Cairns	Recipe Control
44993	Morayfield	Seal Integrity
44998	North Lakes	Seal Integrity
45003	Morayfield	Seal Integrity
45005	Morayfield	Seal Integrity
45006	Morayfield	Seal Integrity
45006	Morayfield	Seal Integrity
45008	Morayfield	Seal Integrity
45009	Morayfield	Seal Integrity
45010	Morayfield	Seal Integrity
45010	Morayfield	Seal Integrity
45011	North Lakes	Seal Integrity
45011	Eatons Hill	Seal Integrity
45011	Morayfield	Seal Integrity
45012	Keperra	Seal Integrity
45015	Cairns	Seal Integrity
45016	Morayfield	Seal Integrity
45017	Morayfield	Seal Integrity
45019	Morayfield	Seal Integrity
45019	Cairns	Seal Integrity
45021	Morayfield	Seal Integrity
45025	Eatons Hill	Seal Integrity
45024	Cairns	Seal Integrity
45023	Morayfield	Seal Integrity
45024	Morayfield	Seal Integrity
45027	Cairns	Seal Integrity
45029	Eatons Hill	Seal Integrity
45033	Keperra	Seal Integrity
45050	Morayfield	Seal Integrity
45047	Morayfield	Seal Integrity
45048	Morayfield	Seal Integrity
45048	Morayfield	Seal Integrity
45049	Morayfield	Seal Integrity
45050	Morayfield	Seal Integrity
45052	Morayfield	Seal Integrity
45052	Morayfield	Seal Integrity
45052	Keperra	Missing stock
45053	Morayfield	Seal Integrity
45055	Morayfield	Seal Integrity
45056	Morayfield	Seal Integrity
45056	Morayfield	Seal Integrity
45057	Morayfield	Seal Integrity
45058	Morayfield	Seal Integrity
45058	Eatons Hill	Seal Integrity
45061	Eatons Hill	Seal Integrity
45061	Morayfield	Seal Integrity
45061	Morayfield	Seal Integrity
45063	Morayfield	Seal Integrity
45064	Eatons Hill	Seal Integrity
45068	Morayfield	Seal Integrity
45070	Morayfield	Seal Integrity
45071	Morayfield	Seal Integrity
45072	Annerley	Seal Integrity
45072	Keperra	Seal Integrity
45073	Morayfield	Seal Integrity
45073	Morayfield	Seal Integrity
45075	Morayfield	Seal Integrity
45075	Morayfield	Seal Integrity
45076	Morayfield	Seal Integrity
45078	Morayfield	Seal Integrity
45081	Eatons Hill	Seal Integrity
45083	Annerley	Seal Integrity
45085	Morayfield	Seal Integrity
45085	Morayfield	Seal Integrity
45087	Morayfield	Seal Integrity
45090	Morayfield	Seal Integrity
45090	Keperra	Seal Integrity
45091	Ballina	Seal Integrity
45091	Morayfield	Seal Integrity
45092	Morayfield	Seal Integrity
45094	Morayfield	Seal Integrity
45095	Morayfield	Seal Integrity
"""

# Splitting by lines and then by tabs
lines = data_str.strip().split('\n')
for line in lines[1:]:  # we skip the first line because it's headers
    fields = line.split('\t')

    if len(fields) == 3:  # we ensure that each line has 3 fields
        date_raised.append(int(fields[0]))
        location.append(fields[1])
        complaint_type.append(fields[2])
    else:
        print(f"Skipping line: {line}")

# Now, we should have equal-length lists
assert len(date_raised) == len(location) == len(complaint_type), "All lists must have the same length."

# Creating a pandas DataFrame from the lists
complaints_df = pd.DataFrame({'Date Raised': date_raised, 'Location': location, 'Complaint Type': complaint_type})





def plot_map(date_filter, complaint_filter):
    # Filter complaints dataframe
    filtered_complaints = complaints_df[(complaints_df['Date Raised'] == date_filter) &
                                        (complaints_df['Complaint Type'] == complaint_filter)]
    complaint_counts = filtered_complaints['Location'].value_counts()

    # Add complaint count to the suburbs dataframe
    suburbs_df['Complaint Count'] = suburbs_df['Suburb Name'].map(complaint_counts)

    # Normalize complaint count for color mapping
    suburbs_df['Complaint Count (Normalized)'] = (suburbs_df['Complaint Count'] -
                                                suburbs_df['Complaint Count'].min()) / (
                                                suburbs_df['Complaint Count'].max() -
                                                suburbs_df['Complaint Count'].min())

    # Create a map centered around Australia
    australia_map = folium.Map(location=[-24.396308, 133.745896], zoom_start=4)

    # Add a marker for each suburb
    for idx, row in suburbs_df.iterrows():
        color = plt.cm.Reds(row['Complaint Count (Normalized)'])  # Use matplotlib color map
        color = matplotlib.colors.to_hex(color)  # Convert color to hex
        folium.Marker(
            location=[row['Latitude'], row['Longitude']],
            popup=f"Suburb: {row['Suburb Name']}<br>Complaint Count: {row['Complaint Count']}",
            icon=folium.Icon(color=color)
        ).add_to(australia_map)

    # Display the map
    display(australia_map)

# Define interactive widgets
date_filter = widgets.Select(options=complaints_df['Date Raised'].unique())
complaint_filter = widgets.Select(options=complaints_df['Complaint Type'].unique())

# Interact function binds the function and the widgets together
widgets.interact(plot_map, date_filter=date_filter, complaint_filter=complaint_filter)

interactive(children=(Select(description='date_filter', options=(44963, 44964, 44978, 44979, 44981, 44982, 449…

<function __main__.plot_map(date_filter, complaint_filter)>